### 주택가격 예측 문제: 회귀 예
- Boston Housing Price 데이터세트
- 1970년 중반의 보스톤 외곽 주택의 평균 가격을 예측하는 문제.  재산세율, 범죄율등의 데이터가 집값을 예측하는데 사용됨.
- 데이터세트 506 샘플 중 404개가 훈련 용, 102개가 테스트 용으로 구분
- 13개의 feature 들 (예, 범죄율)이, 각각 다른 범위를 가지고 있음. 어떤 feature 는 0과1 사이의 비율을, 어떤 feature 는 1과 12사이의 범위 등등

- Keras 에 내장된 Boston housing 데이터세트를 로드

In [ ]:
from tensorflow.keras.datasets import boston_housing
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

In [ ]:
train_data.shape

- 404 training sample 과 102 test sample 을 가지고 있음
- 각각 13개의 숫자로 표현된 feature 들 예를들어, 1인당 소득별 범죄율, 거주를 위한 평균 방의 수, 하이웨이로의 접근성등을 보유

In [ ]:
test_data.shape

- target 은 자가 소유의 평균 주택 값을 표시 단위는 천달러

In [ ]:
train_targets

### 데이터 준비 
- 피쳐(행)들의 범위가 제 각각 이어서 feature-wise normalization 을 시행
- 각각의 feature 값과 평균과의 차를 구한다음, 표준 편차로 나누어 줌  (x - $\mu) $/ $\delta$

In [ ]:
mean = train_data.mean(axis=0)
train_data -= mean                 # train_data 와 평균과의 차이 를 구함
std = train_data.std(axis=0)
train_data /= std                  # 이를 standard deviation 과 나누어 줌
test_data -= mean
test_data /= std

### 모델 구축
- 샘플의 갯수가 적기 때문에 64 unit 을 가진 2개의 중간 층을 사용
- training 데이터가 적을수록, overfitting 가능성이 커지나, 모델을 작게 가져가서 이러한 문제를 완화

- 모델 정의  
마지막 층이 단일 unit 을 가진 activation 이 없는 층으로 정의. 이것은 linear layer 로 전형적인 scalar regression 의 정의
즉 단일의 계속적인 값을 예측하는 회귀 문제임  
마지막 층이 순전히 선형이므로, 어떤 범위의 값이든 학습하도록 허용된다.  
loss 함수로 mean squre error 를 선택하였는데, 회귀 문제에서 널리 쓰이는 함수  
훈련중의 측정지수로 mean absolute error 를 측정하는데, 실제값과 예측값과의 차이를 표시한다. 만약 0.5 값이 나오면 500$이 차이라는 뜻

In [ ]:
def build_model():
    model = keras.Sequential([
        layers.Dense(64, activation="relu"),
        layers.Dense(64, activation="relu"),
        layers.Dense(1)
    ])
    model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
    return model

### K-fold cross validation
- 데이터세트가 작아서 validation data 가 100여개 수준
- 이 경우, 어떤 데이터가 validation 으로 속하는 가에 대한 variance 가 validation score 가 많은 영향을 끼침
- 이러한 상황에, K-fole cross validation 이 best practice 가 될 수 있음
- 예를 들어 3-fold cross validation 라면 가용 데이터를 3개로 복제하고 또 각각을 3개로 분할하여  
- 첫번째 fold 의 첫번째 파티션을 validation (이 경우, 두번째, 세번째 파티션을 training) 으로, 두번째 fold 의 두번째 파티션을 valiation, 세번째 fold 의 세번째 파티션을 validation 으로 훈련하여 이 셋을 평균 낸 점수를 사용하는 것이다

In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

k = 4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []
for i in range(k):
    print(f"Processing fold #{i}")
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0)
    model = build_model()
    model.fit(partial_train_data, partial_train_targets,
              epochs=num_epochs, batch_size=1, verbose=0)
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)

In [ ]:
all_scores

-4개 fold 의 평균값을 구하면

In [ ]:
np.mean(all_scores)

- 주택 값의 범위가 10,000 dollar 에서  50,000 dollar 범위 인것을 감안하면, 2,600$의 차이가 보이는 것은 적은 숫자가 아니다.


- 500 epoch 로 훈련을 시켜보자
- 모델의 훈련 상황을 모니터링하기위해 history 를 이용하여 epoch 당 validation score 를 저장하도록 함

In [ ]:
num_epochs = 500
all_mae_histories = []
for i in range(k):
    print(f"Processing fold #{i}")
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0)
    model = build_model()
    history = model.fit(partial_train_data, partial_train_targets,
                        validation_data=(val_data, val_targets),
                        epochs=num_epochs, batch_size=1, verbose=0)
    mae_history = history.history["val_mae"]
    all_mae_histories.append(mae_history)


Processing fold #1


- Building the history of successive mean K-fold validation scores

In [ ]:
average_mae_history = [
    np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]

- Validation score 를 plot 

In [ ]:
 plt.plot(range(1, len(average_mae_history) + 1), average_mae_history)
plt.xlabel("Epochs")
plt.ylabel("Validation MAE")
plt.show()

**처음 10개 데이터 포인트를 제거하고 자연스러운 validation score 를 plot

In [ ]:
def smooth_curve(points, factor=0.9):
    smoothed_points = []
    for point in points:
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point * (1 - factor))
        else:
            smoothed_points.append(point)
    return smoothed_points
smooth_mae_history = smooth_curve(average_mae_history[10:])
plt.plot(range(1, len(smooth_mae_history) + 1), smooth_mae_history)
plt.xlabel("Epochs")
plt.ylabel("Validation MAE")
plt.show()

### Training Final Model



In [ ]:
model = build_model()
model.fit(train_data, train_targets,
          epochs=80, batch_size=16, verbose=0)
test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)

In [ ]:
test_mae_score


### 새로운 데이터를 대상으로 예측

In [ ]:
predictions = model.predict(test_data)
predictions[0]